### PROJECT_4 : создание рекомендательной системы для онлайн-школы MasterMind.
    Стек инструментов : Python.Pandas, SQL.Metabase, PostrgeQSL.PgAgent, мат.статистика.А/B-Тест
    Задача прямая: создать рекомендательную систему для выявления интереса пользователей (каждому курсу будут рекомендованы ещё два)
    Задача косвенная: увеличение среднего чека покупки Курсов
    Результат в следующем виде: таблица рекомендаций /  sql-скрипты / python-код / A/B-тест анализа результатов


ОПИСАНИЕ ДАННЫХ
- carts [таблица пользовательских корзин] sql-анализ
    - Promo Code ID — ID промокода, если он есть
    - Purchased At — дата оплаты
    - User ID — ID пользователя
    - Created At — дата создания корзины
    - Updated At — дата последнего обновления информации
    - ID — идентификатор корзины
    - State — состояние оплаты

- art_items [таблица курсов, которые пользователи добавили в корзину] - sql-анализ
    - Created At — дата создания события
    - Resource Type — тип продукта
    - Resource ID — ID курса
    - Cart ID — идентификатор корзины
    - Updated At — дата последнего обновления информации
    - ID — идентификатор операции

- data_sql.csv [таблица id пользователей и id курсов которые они приобрели]
     - user_id — ID пользователя
     - resource_id — ID курса

In [13]:
import pandas as pd
import numpy as np
import collections
from collections import Counter
import warnings
import matplotlib.pyplot as plt
import psycopg2
import psycopg2.extras
import seaborn as sns

# 1. Выгрузка данных  из SQL
def getUser_idAndCoursesList():
    query = '''
 -- CTE x: таблица с успешными покупками курсов
With x AS(
SELECT c.user_id,
       c.state,
       it.resource_type,
       it.resource_id
FROM final.carts c JOIN final.cart_items it ON c.id = it.cart_id
WHERE it.resource_type = 'Course'
AND state = 'successful'
),

-- CTE y: таблица клиентов, купивших более одного  курса и количеством покупок
y AS(
SELECT x.user_id, count(x.resource_id)
FROM x
GROUP BY 1
HAVING count(distinct x.resource_id) > 1
),

-- CTE u: таблица  id клиентов для дальнейшего сравнения  и выборки
u AS(
SELECT user_id FROM y
)

-- таблица клиентов, которым соответствует купленный курс  (34074 строк)
SELECT u.user_id, x.resource_id
FROM x
RIGHT JOIN u ON x.user_id=u.user_id
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

# При невозможности выгрузить данные из базы можно воспользоваться файлом .csv
# df = pd.read_csv('data_sql.csv')


df = pd.DataFrame(getUser_idAndCoursesList()) # из данных запроса создает таблицу df
df = df.drop_duplicates(ignore_index=True)    # уберем дубоированные покупки одного курса одним пользователем


# таблица  пользователей и купленных ими курсов
# df: [34074 rows × 2 cols]
display(df)

,user_id,resource_id
0,1010882,490
1,1010802,514
2,1120298,514
3,750528,552
4,191574,523
...,...,...
34069,1164493,742
34070,1249905,566
34071,718824,752
34072,1148335,490


#### 2. Подготовка словаря пар курсов, которые встречаются в покупках у клиентов

In [ ]:
# Для каждого клиента в df собраны все купленные им курсы
# Зеркальные пары убираем сортировкой
user_groups = df.groupby('user_id')['resource_id'].apply(list).reset_index()
user_groups['sorted_resource'] = user_groups['resource_id'].apply(lambda x: sorted(x)) # display(user_groups)


# Узнаем сколько различных пар курсов встречаются вместе в покупках клиентов
# используем вспомогательный list(), itertools.combinations() и Counter()
import itertools
from itertools import combinations

courses = []
for i in user_groups['sorted_resource']:
    for j in itertools.combinations(i, 2):
        courses.append(j)  # display(courses)


# Количество уникальных пар в списке - 3989 уникальные пары
couplesCount = Counter(courses)
couplesUnique = list(couplesCount)  # display(len(c))


# Cловарь пар курсов:  3989 (key - пара, value - количество повторений)
couples_most_common = couplesCount.most_common()
couples_dict  = dict(couples_most_common)
couples_dictC = Counter(couples_dict) # display(couples_dictC)


# Таблица уникальных id курсов, за исключением курса, который ни разу не продавался
cources_and_purchase = df.groupby('resource_id')['user_id'].count().reset_index()
all_cources = cources_and_purchase['resource_id']



### Минимальная граница повторений, допустимых для рекомендаций

- Разделим на квантили количества повторений
- Пары с q=0.25 (повторений нет) и q=0.5 (3 повторения) рекоментовать не будем
- Для рекомендации оставим курсы с q=0.75 (9 повторений)
- Для курсов, не прошедших нимальгую границу: будем рекомендовать самую популярную пару кусов (551, 566): 797 повторений пары
     - курс с id 551 был куплен 2935 раз,  566 - 2342 раза
    

In [26]:
# Запишем перечень пар и количество их повторений во вспомогательный df
df1 = pd.DataFrame({'couples': couples_most_common}, index=None)
df1['count_repeat'] = df1['couples'].apply(lambda x: x[1])     # количество
quantiles = df1['count_repeat'].quantile(q=[0.25, 0.5, 0.75])  # display('quantiles', quantiles)

quantiles.reset_index()

,index,count_repeat
0,0.25,1.0
1,0.50,3.0
2,0.75,9.0


### 3. Подготовка Таблицы рекомендаций:

In [32]:
# Функция рекомендаций на вход принимает номер курса, возврвщвет 2 курса, с max числом повторенй
# Цикл по всем уникальным курсам и запись результатов в датафрейм.
def Recommend(x):
    cource_list = []
    for i in couples_dict.keys():
        if (i[0] == x and couples_dictC[i] >= 9):          # условие: 1е знач. ключа соответствует номеру курса, повторений >=9
           cource_list.append((i[1], couples_dictC[i]))    # ключ и значение в список : курс-рекомндация
        if i[1] == x and couples_dictC[i] >= 9:            # условие: 2е знач.  ключа (рекомндация) соответствует номеру курса, повторений >=9
           cource_list.append((i[0], couples_dictC[i]))    # ключ и значение в список :
        if i[0] == x and couples_dictC[i] < 9:
           cource_list.append((551,0))                     # (551, 566): 797 - самая популярная пара кусов, рекомендуем их альтернативно
        if i[1] == x and couples_dictC[i] <9:
           cource_list.append((566,1))
    sorted_list = sorted(cource_list, key=lambda x:x[0], reverse = True) # сортировка от большего к меньшему числу повторений
    return sorted_list[:2]


# display(all_cources.apply(lambda x: Recommend(x))) # исходный вид выгрузки
# пустой df для таблицы
recommend_df = pd.DataFrame(columns = ['courses_ids', 'recommend_1', 'recommend_2'])


# Заполнение df рекомендаций
recommend_df['courses_ids'] = df['resource_id'].unique()
recommend_df['recommend_1']= all_cources.apply(lambda x: Recommend(x)[0][0])
recommend_df['recommend_2'] = all_cources.apply(lambda x: Recommend(x)[1][0])


# Выгрузка таблицы рекомендаций в excel
recommend_df.to_excel('recommend_df.xlsx', sheet_name='table', index=False)


recommend_df.sort_values(by='courses_ids')

,courses_ids,recommend_1,recommend_2
17,356,1161,1145
16,357,1185,1140
40,358,1186,1144
5,359,1138,1125
33,360,1103,1100
...,...,...,...
81,1188,908,566
120,1198,1139,1099
124,1199,566,566
119,1200,1125,912


### РЕЗЮМИРУЕМ

- На этапе анализа в sql были изучены исходные данные:
в  таблицах  содержатся данные по продажам курсов за 2017-2018 гг
    - 127 различных  курса у школы
    - 49 006 клиентов покупали курсы
    - 12 656  клиентов купили больше  1го  курса (это 25 % всех покупателей курсов)
    - 1.44 - среднее число купленных курсов на одного клиента
- Из БД была выгружена таблица всех пользователей, купивших курс в паре с их покупками (data_sql.csv)

- Были обработаны данные средствами Python и создана таблица с рекомендациями (recommend_df.xlsx) каждому курсу в базе рекомендованы еще два.

- Рекомендательная система была построена на основании тех курсов которые клиент уже купил. Просмотрены курсы, чаще всего встречающиеся в паре с купленным и выбраны два с самой высокой частотой повторений.

- Минимальной границей значимости было принято число повторений, равное квантилю = 0.75 (в нашем датасете это 9 повторений). Для курсов, не прошедших данный порог, рекомендован альтернативный вариант - 2 самых популярных по повторениям и покупкам курса

- Для запуска A/B-теста рассчитан минимальный размер выборки для получения статистически значимого результата, равный 7 866 человек

- Проведено тестирование гипотезы - получение значимого результата при запуске рекомендательной системы. Все клиенты случайным образом делились на контрольную и тестовую группы:
тестовой группе показываются рекомендации, а контрольной — нет.     
    - В контрольной группе оказалось 8732 клиента, из них 293 купили больше одного курса
    - В тестовой — 8847 клиентов, из них 347 купили больше одного курса

- Вариант с рекомендациями показал статистически значимо лучший результат, нужно реализовывать его для всех клиентов.


